In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
def nDCG(relevances: np.ndarray, positions: np.ndarray) -> float:
    """Compute Normalized Discounted Cumulative Gain based on:
    - relevances: Numpy Array containing DCG Relevances (5 if booked, 1 if clicked)
    - positions: Numpy Array containing Positions (The display order) """
    
    positions_normalized = np.argsort(positions)
    relevances_ordered = relevances[positions_normalized]
    relevances_sorted = np.sort(relevances)[::-1]
    
    gain = 2 ** relevances_ordered - 1
    ideal_gain = 2 ** relevances_sorted - 1
    
    discount = np.log2(np.arange(len(positions)) + 2)
    
    DCG = np.sum(gain / discount)
    IDCG = np.sum(ideal_gain / discount)
    
    return DCG / IDCG


def nDCG_mean(dataframe: pd.DataFrame) -> float:
    """
    Calculate Mean Normalized Discounted Cumulative Gain on DataFrame
    DataFrame must have fields: [srch_id, relevance, position]
    """
    
    nDCG_sum = 0.0
    
    searches = dataframe.groupby('srch_id')
    
    for name, search in searches:
        nDCG_sum += nDCG(search.relevance.values, search.position.values)
    return nDCG_sum / len(searches)

In [3]:
# Load That Shit (does take a while)
df = pd.read_csv("data/training_set_VU_DM_2014.csv")

In [4]:
# Add Relevance Column to DataFrame
relevance = np.zeros(len(df))
relevance[df['click_bool'] == 1] = 1
relevance[df['booking_bool'] == 1] = 5

df['relevance'] = relevance

In [5]:
# Reorganize DataFrame (a.k.a. throw out competitor info): handy for tweaking/overview?
df = df[[
    # ID
    'srch_id',
    
    # Labels (to predict)
    'position',
    'relevance',
    'click_bool',
    'booking_bool',
    
    # Per Seach Features
    'date_time',
    'site_id',
    'srch_destination_id',
    'srch_length_of_stay',
    'srch_booking_window',
    'srch_adults_count',
    'srch_children_count',
    'srch_room_count',
    'srch_saturday_night_bool',
    'srch_query_affinity_score',
    'orig_destination_distance',
    
    # Property Features
    'price_usd',
    'promotion_flag',
    'prop_country_id',
    'prop_id',
    'prop_starrating',
    'prop_review_score',
    'prop_brand_bool',
    'prop_location_score1',
    'prop_location_score2',
    'prop_log_historical_price',
    
    # Visitor Features
    'visitor_location_country_id',
    'visitor_hist_starrating',
    'visitor_hist_adr_usd',
]]

# Data Cleaning

In [6]:
# Clean NaN's (The Pandas Way)
df.srch_query_affinity_score = df.srch_query_affinity_score.fillna(df.srch_query_affinity_score.mean())
df.orig_destination_distance = df.orig_destination_distance.fillna(df.orig_destination_distance.mean())

df.prop_location_score2 = df.prop_location_score2.fillna(df.prop_location_score2.min())

df.visitor_hist_adr_usd = df.visitor_hist_adr_usd.fillna(-1)
df.visitor_hist_starrating = df.visitor_hist_starrating.fillna(-1)
df.prop_review_score = df.prop_review_score.fillna(df.prop_review_score.min())

In [7]:
# Check Percentage of NaN's per Column

for column in df.columns:
    print("{:50s}{:4.1%} {}".format(column, np.mean(df[column].isnull()), "NaN" if df[column].isnull().any() else ""))

srch_id                                           0.0% 
position                                          0.0% 
relevance                                         0.0% 
click_bool                                        0.0% 
booking_bool                                      0.0% 
date_time                                         0.0% 
site_id                                           0.0% 
srch_destination_id                               0.0% 
srch_length_of_stay                               0.0% 
srch_booking_window                               0.0% 
srch_adults_count                                 0.0% 
srch_children_count                               0.0% 
srch_room_count                                   0.0% 
srch_saturday_night_bool                          0.0% 
srch_query_affinity_score                         0.0% 
orig_destination_distance                         0.0% 
price_usd                                         0.0% 
promotion_flag                                  

# Feature Engineering

In [8]:
# Extract Month/Day/Hour from Datetime
dates = pd.DatetimeIndex(df['date_time'])

df['month'] = dates.month
df['day'] = dates.day
df['hour'] = dates.hour

In [9]:
def normalize(df: pd.DataFrame, feature: str, with_respect_to: str):
    ft = df[feature]
    ft_mean = df.groupby(with_respect_to)[feature].transform('mean')
    ft_std = df.groupby(with_respect_to)[feature].transform('std')
    
    ft_std[ft_std == 0] = 1    
    return (ft - ft_mean) / ft_std


#Normalize Numerical Features with Respect to Search ID, Property ID and Destination ID and add those to the DataFrame
for norm_group in ['srch_id', 'srch_destination_id', 'srch_booking_window', 'prop_id', 'prop_country_id', 'month',]:
    for feature in ['price_usd', 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price']:
        df["{}_{}".format(feature, norm_group)] = normalize(df, feature, norm_group)

In [10]:
for feature in ['price_usd', 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price']:
    df["{}_prop_id".format(feature)] = df["{}_prop_id".format(feature)].fillna(df["{}_prop_id".format(feature)].min())

In [11]:
df['price_ranking'] = df.groupby('srch_id')['price_usd'].transform(lambda x: np.argsort(x) / len(x))

# Test/Train Split

In [12]:
# Sample Random Fraction of Searches of DataFrame (To Speed Up Shit)
# (This does take a while though...)
RANDOM_FRACTION = 0.5
selection = df.groupby('srch_id').filter(lambda x: np.random.uniform() < RANDOM_FRACTION)

In [13]:
# Select Subset Of Best Features (And Include Labels, of course)
selection = selection[[
    # ID
    'srch_id',
    
    # Labels (to predict)
    'position',
    'relevance',
    'click_bool',
    'booking_bool',
    
    'price_usd_srch_id',
    'prop_location_score2_srch_id',
    'prop_log_historical_price_srch_id',
    'prop_starrating',
    'prop_location_score1_prop_country_id',
    'prop_location_score2_srch_booking_window',
    'srch_length_of_stay',
    'prop_review_score_srch_id',
    'price_usd_prop_id',
    'srch_booking_window',
    'price_usd',
    'prop_location_score2_prop_id',
    'promotion_flag',
    'srch_adults_count',
    'prop_location_score1',
    'prop_review_score_srch_booking_window',
    'prop_location_score1_month',
    'srch_room_count',
    'visitor_hist_adr_usd',
    'prop_location_score2_srch_destination_id',
    'price_ranking',
    'price_usd_srch_booking_window',
    'prop_location_score2_month',
    'srch_query_affinity_score',
    'prop_review_score_prop_country_id',
    'prop_location_score1_srch_booking_window',
    'prop_review_score_month',
    'prop_log_historical_price_srch_destination_id',
    'srch_children_count',
    'price_usd_prop_country_id',
    'prop_review_score_srch_destination_id',
    'prop_location_score2',
    'prop_location_score1_srch_id',
    'prop_location_score2_prop_country_id',
    'prop_log_historical_price_prop_id',
    'price_usd_srch_destination_id',
    'prop_log_historical_price_month',
]]

In [14]:
# Split Train and Test from random selection, again per Search ID
TEST_TRAIN_SPLIT = 0.9

unique_search_ids = np.unique(selection.srch_id)
mask = np.random.uniform(0, 1, len(unique_search_ids)) < TEST_TRAIN_SPLIT

train = selection[selection.srch_id.isin(unique_search_ids[mask])]
test = selection[selection.srch_id.isin(unique_search_ids[~mask])]

# Downsample Training Set Negative Instances (Tip From Owen)
DOWNSAMPLE = 0.1
train = pd.concat([train[train.relevance > 0], train[train.relevance == 0].sample(frac=DOWNSAMPLE)])

# Split Into Features and Labels
train_features = train.loc[:, 'price_usd_srch_id':]
train_labels = train['relevance']

test_features = test.loc[:, 'price_usd_srch_id':]
test_labels = test['relevance']

print(len(train), len(test))

313529 251111


# Machine Learning

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

# Train Regressor on Train Features w.r.t. Train Labels (a.k.a. Relevances)
classifier = GradientBoostingRegressor(n_estimators=500, verbose=1)
classifier.fit(train_features, train_labels)

# Predict Relevances
predict_labels = classifier.predict(test_features)

# Through Data to Calculate Score in new DataFrame
# Note that predicted position = - predicted relevance
result = pd.DataFrame({
    'srch_id': test.srch_id,
    'relevance': test.relevance,
    'position': -predict_labels})
print("Prediction:", nDCG_mean(result))

# Plot Feature Importances Graph
fig = plt.figure(figsize=(10, 10))
fi_sort = np.argsort(classifier.feature_importances_)
plt.barh(np.arange(len(test_features.columns)), classifier.feature_importances_[fi_sort],
       tick_label=test_features.columns[fi_sort])

      Iter       Train Loss   Remaining Time 
         1           3.7920           11.22m
         2           3.7492           11.47m
         3           3.7106           11.65m
         4           3.6808           11.51m
         5           3.6532           11.23m
         6           3.6306           11.04m
         7           3.6100           11.15m
         8           3.5933           11.11m
         9           3.5788           10.91m
        10           3.5643           10.85m
        20           3.4766           10.43m
        30           3.4269           10.11m
        40           3.3902            9.77m
        50           3.3637            9.48m
        60           3.3464            9.13m
        70           3.3334            8.90m
        80           3.3217            8.67m
        90           3.3122            8.39m
       100           3.3047            8.10m
       200           3.2582            5.99m
       300           3.2328            3.90m
       40